In [2]:
# imported mysql.connector to be able to connect the database
import mysql.connector as mysql

# data management imports
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
# load data in DataFrame
df = pd.read_csv("../Interactive Map/region_data.csv")

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df

,berry,observation_date,main_tree_type,forest_development_class,x,y,year,month,day,week_number,monthday,year-month-day,xy,lat,lon,latlon,region
0,bilberry,2018-07-07,pine,old,410000,7040000,2018,7,7,27,707,2018-07-07,410000-7040000,63.476942,25.193479,63.476942248701-25.193478894290-2018-07-07-bil...,Central Finland
1,lingonberry,2018-08-06,pine,old,410000,7040000,2018,8,6,32,806,2018-08-06,410000-7040000,63.476942,25.193479,63.476942248701-25.193478894290-2018-08-06-lin...,Central Finland
2,cloudberry,2018-07-07,pine,clearing,410000,7040000,2018,7,7,27,707,2018-07-07,410000-7040000,63.476942,25.193479,63.476942248701-25.193478894290-2018-07-07-clo...,Central Finland
3,bilberry,2022-07-16,pine,old,410000,7040000,2022,7,16,28,716,2022-07-16,410000-7040000,63.476942,25.193479,63.476942248701-25.193478894290-2022-07-16-bil...,Central Finland
4,lingonberry,2022-08-28,pine,old,410000,7040000,2022,8,28,34,828,2022-08-28,410000-7040000,63.476942,25.193479,63.476942248701-25.193478894290-2022-08-28-lin...,Central Finland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,lingonberry,2020-08-12,spruce,seedling,500000,7080000,2020,8,12,33,812,2020-08-12,500000-7080000,63.847317,27.000016,63.847316635241-27.000015662314-2020-08-12-lin...,Northern Savo
852,cloudberry,2020-07-22,pine,old,500000,7080000,2020,7,22,30,722,2020-07-22,500000-7080000,63.847317,27.000016,63.847316635241-27.000015662314-2020-07-22-clo...,Northern Savo
853,cloudberry,2020-07-15,pine,seedling,560000,7140000,2020,7,15,29,715,2020-07-15,560000-7140000,64.380480,28.243717,64.380480243615-28.243717349216-2020-07-15-clo...,Kainuu
854,cloudberry,2020-07-18,pine,clearing,400000,7080000,2020,7,18,29,718,2020-07-18,400000-7080000,63.833034,24.967397,63.833033637422-24.967397132992-2020-07-18-clo...,Northern Ostrobothnia


<h3>data to the backend</h3>
<h3></h3>

In [3]:
# connecting to MySQL server
mydb = mysql.connect(
    host="154.49.137.221",
    user="Nikita",
    password="CheeseTax",
    database="berry_nice",
    auth_plugin="mysql_native_password"
)

# select the correct database here
mycursor = mydb.cursor()


In [4]:
# creates the database
mycursor.execute("CREATE DATABASE IF NOT EXISTS berry_nice")

In [5]:
# selecting database
mycursor.execute("USE berry_nice")


In [6]:
# creating the table
# mycursor.execute("CREATE TABLE IF NOT EXISTS berries (log_id INT, berry VARCHAR(255), observation_date Datetime, main_tree_typee VARCHAR(255), forest_development_class VARCHAR(255),x_cord INT, y_cord INT, year INT, month INT, day INT, week_number INT, PRIMARY KEY (log_id))")

mycursor.execute("CREATE TABLE IF NOT EXISTS berries (log_id INT, berry VARCHAR(255), main_tree_type VARCHAR(255), forest_development_class VARCHAR(255),lat FLOAT(8,6), lon FLOAT(8,6),observation_date DATE, year INT, month INT, day INT, week_number INT, latlon VARCHAR(255), region VARCHAR(255), html_id VARCHAR(10), PRIMARY KEY (log_id))")


In [7]:
# looping the data to the mysql 

# list of all items in excel sheet
val = []

# looping trough the dataframe
# getting the index number of the row for getting the id for the tables
# getting the row using iterrows
for index, row in df.iterrows():
    log_id = 10000 + index

    # data that will be inserted to the mysql
    data = f"{log_id}, {row['berry']}, {row['main_tree_type']}, {row['forest_development_class']}, {row['lat']}, {row['lon']}, {row['observation_date']}, {row['year']}, {row['month']}, {row['day']}, {row['week_number']}, {row['latlon']}, {row['region']}"

    # Convert the string to a tuple
    data_tuple = tuple(data.split(", "))

    # appending the data to the value list
    val.append(data_tuple)


# insert statement for the mysql
sql = "INSERT INTO berries (log_id, berry, main_tree_type,forest_development_class ,lat ,lon ,observation_date ,year, month, day, week_number, latlon, region) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

# excecute the command sql, add the values in list val
mycursor.executemany(sql, val)

# add region codes
region_codes = {'Lapland': 'FI-10T', 'Northern Ostrobothnia': 'FI-14T', 'Kainuu': 'FI-05T', 'Southern Savonia': 'FI-04T', 'Central Finland': 'FI-08T', 'North Karelia': 'FI-13T', 'Pirkanmaa': 'FI-11T', 'Southern Ostrobothnia': 'FI-03T', 'Northern Savo': 'FI-15T', 'Varsinais-Suomi': 'FI-19T', 'Uusimaa': 'FI-18T', 'South Karelia': 'FI-02T', 'Paijanne Tavastia': 'FI-16T', 'Central Ostrobothnia': 'FI-07T', 'Haeme': 'FI-06T', 'Satakunta': 'FI-17T', 'Kymenlaakso': 'FI-09T', 'Ostrobothnia': 'FI-12T'}

for x, y in region_codes.items():
    sql = "UPDATE berries SET html_id = %s WHERE region = %s"
    val = (y, x)
    mycursor.execute(sql, val)

# commit
mydb.commit()


# print the amount of rows inserted
print(mycursor.rowcount, "record inserted.")


856 record inserted.


In [8]:
# Creating region_counts table for Interactive Map
mycursor.execute("CREATE TABLE IF NOT EXISTS region_counts (region VARCHAR(255) NOT NULL, count INT UNSIGNED NOT NULL DEFAULT 0, html_id VARCHAR(10), bilberry INT UNSIGNED NOT NULL DEFAULT 0, lingonberry INT UNSIGNED NOT NULL DEFAULT 0, cloudberry INT UNSIGNED NOT NULL DEFAULT 0, year INT, PRIMARY KEY (region));")

In [9]:
counts = df['region'].value_counts()
counts

Lapland                  161
Northern Ostrobothnia    142
Kainuu                    85
Southern Savonia          70
Central Finland           50
North Karelia             46
Pirkanmaa                 41
Southern Ostrobothnia     39
Northern Savo             35
Varsinais-Suomi           29
Uusimaa                   25
South Karelia             23
Paijanne Tavastia         23
Central Ostrobothnia      21
Haeme                     19
Finnmark Fylke            17
Satakunta                 16
Republic of Karelia        8
Kymenlaakso                4
Ostrobothnia               2
Name: region, dtype: int64

In [10]:
# Add region counts
for x, y in counts.items():
    sql = "INSERT INTO region_counts (region, count) VALUES (%s, %s)"
    val = (x, y)
    mycursor.execute(sql, val)


In [11]:

# remove unnecessary rows and add a missing region
mycursor.execute("DELETE FROM region_counts WHERE region LIKE 'Finnmark Fylke'")
mycursor.execute("DELETE FROM region_counts WHERE region LIKE 'Republic of Karelia'")
mycursor.execute("INSERT INTO region_counts (region, html_id) VALUES ('Alands landsbygd', FI-01T)")

In [12]:
# add region codes
region_codes = {'Lapland': 'FI-10T', 'Northern Ostrobothnia': 'FI-14T', 'Kainuu': 'FI-05T', 'Southern Savonia': 'FI-04T', 'Central Finland': 'FI-08T', 'North Karelia': 'FI-13T', 'Pirkanmaa': 'FI-11T', 'Southern Ostrobothnia': 'FI-03T', 'Northern Savo': 'FI-15T', 'Varsinais-Suomi': 'FI-19T', 'Uusimaa': 'FI-18T', 'South Karelia': 'FI-02T', 'Paijanne Tavastia': 'FI-16T', 'Central Ostrobothnia': 'FI-07T', 'Haeme': 'FI-06T', 'Satakunta': 'FI-17T', 'Kymenlaakso': 'FI-09T', 'Ostrobothnia': 'FI-12T'}

for x, y in region_codes.items():
    sql = "UPDATE region_counts SET html_id = %s WHERE region = %s"
    val = (y, x)
    mycursor.execute(sql, val)

In [14]:
# get berry counts by region
berries = df['berry'].value_counts()

# get berry name
for berry, count in berries.items():
    sql = "SELECT region, COUNT(*) FROM berries WHERE berry = %s GROUP BY region"
    val = (berry,)
    mycursor.execute(sql, val)
    myresult = mycursor.fetchall()
    myresult = dict(myresult)

    # update berry numbers
    for region, berrycount in myresult.items():
        
        if berry == 'bilberry':
            sql = "UPDATE region_counts SET bilberry = %s WHERE region = %s"
        
        elif berry == 'lingonberry':
            sql = "UPDATE region_counts SET lingonberry = %s WHERE region = %s"
        
        elif berry == 'cloudberry':
            sql = "UPDATE region_counts SET cloudberry = %s WHERE region = %s"
        
        val = (berrycount, region,)
        mycursor.execute(sql, val)

In [15]:
# commit
mydb.commit()

In [16]:

# close
mydb.close()